# IBM Data Science Capstone Project- Battle of the Neighborhoods 

### I will import appropriate libraries to anaylze data in a vectorized manner.

In [1]:
# library to handle data in a vectorized manner
import numpy as np 

# library for data analsysis
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# library to handle requests
import requests 

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# map rendering library
import folium 

print('Libraries Imported')

Libraries Imported


### I will use my credentials to access Foursquare Developer API to begin my requests.

In [2]:
# your Foursquare ID
CLIENT_ID = 'BBXNP5GX0KIWYKMM0TGGM1CRJZL0RFTHWIWXJNYIN4V04PFP' 

# your Foursquare Secret
CLIENT_SECRET = 'ZAVISO3O2PLB2SKHRDNEBJFOXWSUDWXY41CB3POPYQWZW1TL' 

# Foursquare API version
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('Foursquare Verified')

Your credentails:
CLIENT_ID: BBXNP5GX0KIWYKMM0TGGM1CRJZL0RFTHWIWXJNYIN4V04PFP
Foursquare Verified


### I will set up my parameters for my requests to collect data for clustering pizza venues for each city.

In [3]:
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Jersey City, NJ', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ca941735") # PIZZA PLACE CATEGORY ID
    results[city] = requests.get(url).json()
    
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']
print('Request Complete')

Request Complete


### The Foursquare API only gives us the nearest 100 venues in each city, so I will examine the denisty of each requested city visually and record the total number of pizza venues.


In [4]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of pizza venues in {city} = ", results[city]['response']['totalResults'])

Total number of pizza venues in New York, NY =  319
Total number of pizza venues in Chicago, IL =  227
Total number of pizza venues in San Francisco, CA =  169
Total number of pizza venues in Jersey City, NJ =  124
Total number of pizza venues in Boston, MA =  187


### I will use folium to map the clusters of each city.

### New York, NY

In [5]:
maps[cities[0]]

### Chicago, IL

In [6]:
maps[cities[1]]

### San Francisco, CA

In [7]:
maps[cities[2]]

### Jersey City, NJ

In [8]:
maps[cities[3]]

### Boston, MA

In [9]:
maps[cities[4]]

### Visually, we can see that New York, NY and Chicago, IL seem to have the highest density of pizza venues for target locations to set up shop.
### However, if we employ some statistics, we can get the mean location of the pizza venues to ascertain the validity of their actual denisty.
### Afterwards, we  will take the average of the distance of the venues to the mean coordinates to get a more concrete density.

In [10]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean Coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean Coordinates
0.022656966339103884
Chicago, IL
Mean Distance from Mean Coordinates
0.05723120018998456
San Francisco, CA
Mean Distance from Mean Coordinates
0.02782241910063773
Jersey City, NJ
Mean Distance from Mean Coordinates
0.018821184781042995
Boston, MA
Mean Distance from Mean Coordinates
0.03534053436469107


### We will now map the clusters again based on these statistics provided above.

### New York, NY

In [11]:
maps[cities[0]]

### Chicago, IL

In [12]:
maps[cities[1]]

### San Francisco, CA

In [13]:
maps[cities[2]]

### Jersey City, NJ

In [14]:
maps[cities[3]]

### Boston, MA

In [15]:
maps[cities[4]]

### Interesting enough, although New York, NY and Chicago, IL appeared to be denser visually on the first clustering, the "Mean Distance from Mean Coordinates" performed on the second clusters show that Jersey City, NJ would be denser.
### Based on these findings, I would suggest the client to set up their first International Italian Pizza Shop in Jersey City, NJ. 